In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import gdown
from thefuzz import process

In [ ]:
#Downloading data
# Google drive files id
file_id_1 = '1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi'
file_id_2 = '1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG'
file_id_3 = '1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2'

# Google drive url patterns
# Downloading data
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"
url_3 = f"https://drive.google.com/uc?id={file_id_3}"
output_1 = 'data_dv.csv'
output_2 = 'data_ev.csv'
output_3 = 'data_ur.csv'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)
gdown.download(url_3, output_3, quiet=False)

# Reading data
df1 = pd.read_csv(output_1)
df2 = pd.read_csv(output_2)
df3 = pd.read_csv(output_3,delimiter=";")
df1.head()
df2.head()
df3.head()

# Merging data types
df1['disitrict_name'] = df1['disitrict_name'].astype(str)
df2['Disitrict'] = df2['Disitrict'].astype(str)
df2['district_clean'] = df2['Disitrict'].apply(lambda x:process.extractOne(x,df1['disitrict_name'])[0])
df_merged = df1.merge(df2,left_on='disitrict_name',right_on='district_clean',how='inner')
df_merged.head()
df_merged.rename(columns = {'disitrict_name':'district_name'},inplace=True)
df_merged.drop(columns=['Disitrict','district_clean'], inplace=True)
df_merged.dropna(inplace=True)
df_merged.info()
float_list = ['salary','education_expenses']
for i in float_list:
    df_merged[i] = df_merged[i].astype(str)
    df_merged[i] = df_merged[i].apply(lambda x:x.replace(',',''))
    df_merged[i] = df_merged[i].astype(float)
#df_merged.head()


In [5]:
df3.head()
df3.drop(columns= ['Kod','Unnamed: 3'],inplace=True)
df3[['Nazwa','ogółem;2023;[%]']].astype(str,inplace=True)
df3.rename(columns = {'ogółem;2023;[%]':'une_rate'},inplace=True)



,Kod,Nazwa,ogółem;2023;[%],Unnamed: 3
0,201000,Powiat bolesławiecki,"2,8",NaN
1,202000,Powiat dzierżoniowski,"5,6",NaN
2,203000,Powiat głogowski,"6,5",NaN
3,204000,Powiat górowski,"14,2",NaN
4,205000,Powiat jaworski,"9,7",NaN


In [ ]:
#Linear regression OLS method
Y = df_merged['mean_score']
ev_OLS = ['salary','education_expenses','divorces']
X = df_merged[ev_OLS]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())
